In [3]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling fd7b6731c33c...   0% ▕▏    0 B/9.1 GB                  pulling manifest 
pulling fd7b6731c33c...   0% ▕▏    0 B/9.1 GB                  pulling manifest 

In [4]:
!pip install langchain faiss-cpu PyMuPDF requests langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 113.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [6]:
import requests
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOllama
from langchain.chains import RetrievalQA
import os


pdf_path = os.path.dirname(os.path.abspath("Files/content"))
pdf_url = "https://arxiv.org/pdf/2504.15228"
response = requests.get(pdf_url)
with open(pdf_path, 'wb') as f:
  f.write(response.content)

**Initially the chunks_size was 1000 and chunk_overlap was 100 which delivered unsatifactory result**

In [14]:
loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=300)

chunks = splitter.split_documents(docs)


In [15]:
embeddings = OllamaEmbeddings(model="phi4")
vectorstore = FAISS.from_documents(chunks, embeddings)

llm = ChatOllama(model="phi4")
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

In [16]:
question = "what are the results of paper?"
answer = qa.run(question)
print("Ques: ", question)
print("Ans: ", answer)

Ques:  what are the results of paper?
Ans:  The context provided mentions a workshop paper published at ICLR 2025 SSI-FM, which discusses an experiment involving a 'reasoning agent' named o3 mini (OpenAI, 2025). The results show a promising progression in performance. Specifically:

1. **File Editing Benchmark**: 
   - Initially, the agent could only overwrite files, which was slow and inefficient.
   - It quickly developed more efficient "smart" editing tools using diffs and ranges.

2. **Symbol Navigation**:
   - Although detailed results for this are not explicitly mentioned in the context, it implies improvements similar to those observed in file editing.

Overall, these results demonstrate the agent's capability to enhance its performance over time by developing new strategies and tools to solve problems more efficiently. 

For more comprehensive details or specific figures, you would need to refer directly to Figure 3 and Table 1 from the paper.
